# Build Index

In [3]:
import pandas as pd
# Set corpus to the folder of files you want to use
df = pd.read_csv('/home/ec2-user/SageMaker/data/NYT-all_content.csv')

### 1. Load GOIDS

In [4]:
goid = df['GOID'].to_list()
title = df['Title'].to_list()
text = df['Text'].to_list()
date = df['Date'].to_list()

### 2. Load corpus tensors

In [5]:
import sentence_transformers
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd

embedder = SentenceTransformer('/home/ec2-user/SageMaker/all-MiniLM-L12-v2')

/home/ec2-user/SageMaker/.conda/envs/sample-2022.05.25/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def gen_df(i, j, query):
    
    for idx in range(i,j):
    
        filename = '/home/ec2-user/SageMaker/data/NYT_tensor/tensor' + str(idx) + '.pt'
    
        if idx==i:
            corpus_embedding  = torch.load(filename)
        else: 
            corpus_embedding = torch.concat((corpus_embedding, torch.load(filename)), 0)
    print(corpus_embedding.size())
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embedding)[0]
    print(cos_scores.size())
    i = i*100000
    j = j*100000
    #if j==12:
    #    j=12*100000
    df = pd.DataFrame(list(zip(title[i:j], text[i:j], date[i:j], cos_scores.tolist())), columns = ['title', 'text', 'date', 'score'], index = goid[i:j])
    #df = pd.DataFrame(list(zip(title[i:], text[i:], date[i:], cos_scores.tolist())), columns = ['title', 'text', 'date', 'score'], index = goid[i:])
    #else:
    #    j = j*100000
    #    df = pd.DataFrame(list(zip(title[i:j], text[i:j], date[i:j], cos_scores.tolist())), columns = ['title', 'text', 'date', 'score'], index = goid[i:j])
    print(len(df))
    return df

In [7]:
#%timeit
query = 'greener vehicles' #car fuel efficiency #improvement, innovation and advancement in fuel efficiency #
#df = gen_df(0, 124, query)
df1 = gen_df(0,30, query)
df2 = gen_df(30,60, query)
df3 = gen_df(60,90, query)
#df4 = gen_df(90, 124, query)

torch.Size([3000000, 384])
torch.Size([3000000])
3000000
torch.Size([3000000, 384])
torch.Size([3000000])
3000000
torch.Size([3000000, 384])
torch.Size([3000000])
3000000


In [8]:
df4 = gen_df(90, 124, query)

torch.Size([3316207, 384])
torch.Size([3316207])
3316207


In [10]:
df_f = pd.concat([df1, df2, df3, df4])
df_f = df_f.drop_duplicates(subset='text', keep="first")
df_f.to_csv('/home/ec2-user/SageMaker/data/NYT_scores/2023-08-16-NYT-greenervehicles_score.csv')

In [11]:
len(df_f)

12281380

### Random Sampling

In [ ]:
import pandas as pd
df_f = pd.read_csv('/home/ec2-user/SageMaker/data/2023-07-23-fuelefficiency_score.csv')

In [ ]:
df_f

In [ ]:
df_f['date'] = pd.to_datetime(df_f['date'])

In [ ]:
df_f['year'] = df_f['date'].dt.year

In [ ]:
import numpy as np
df_f = df_f.loc[(df_f['score'] >= -0.3) & (df_f['score'] <= 0.3)]

In [ ]:
ex = df_f.sort_values('score').groupby('year').sample(8)

In [ ]:
ex1 = df_f.sort_values('score', ascending=False).groupby('year').head(4)

In [ ]:
random_sample = pd.concat([ex, ex1])

In [ ]:
ex =ex[ex['year']>=1970].sort_values(by='year')

In [ ]:
ex.to_csv('/home/ec2-user/SageMaker/data/random_sample1.csv')